# Data Aggregation and Group Operations

In [2]:
import pandas as pd
import numpy as np

## GroupBy Mechanics

In [2]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
 ....: 'key2' : ['one', 'two', 'one', 'two', 'one'],
 ....: 'data1' : np.random.randn(5),
 ....: 'data2' : np.random.randn(5)})

df

,key1,key2,data1,data2
0,a,one,0.043317,-0.951015
1,a,two,-0.063972,-0.200997
2,b,one,1.269132,-1.339830
3,b,two,-0.181099,-0.434650
4,a,one,-0.749230,-1.952671


In [3]:
grouped = df['data1'].groupby(df['key1']) #Group data1 with the same key1
grouped

In [4]:
grouped.mean() #Calculate mean fro each key

key1
a   -0.256628
b    0.544016
Name: data1, dtype: float64

In [8]:
mean = df['data1'].groupby([df['key1'], df['key2']]).mean() #Mean of each key2 depening on key1
mean.unstack()

key2,one,two
key1,,
a,-0.352956,-0.063972
b,1.269132,-0.181099


In [9]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005   -0.063972
            2006    1.269132
Ohio        2005   -0.068891
            2006   -0.749230
Name: data1, dtype: float64

In [10]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.256628,-1.034894
b,0.544016,-0.887240


In [11]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.352956 -1.451843
     two  -0.063972 -0.200997
b    one   1.269132 -1.339830
     two  -0.181099 -0.434650

In [12]:
df.groupby(['key1', 'key2']).size() #Size of each group

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

## Iterating Over Groups

In [13]:
for name, group in df.groupby('key1'): #Splits up by key1
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  0.043317 -0.951015
1    a  two -0.063972 -0.200997
4    a  one -0.749230 -1.952671
b
  key1 key2     data1    data2
2    b  one  1.269132 -1.33983
3    b  two -0.181099 -0.43465


In [15]:
for (k1,k2), group in df.groupby(['key1', 'key2']): #Split by key1 and key2
    print((k1,k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.043317 -0.951015
4    a  one -0.749230 -1.952671
('a', 'two')
  key1 key2     data1     data2
1    a  two -0.063972 -0.200997
('b', 'one')
  key1 key2     data1    data2
2    b  one  1.269132 -1.33983
('b', 'two')
  key1 key2     data1    data2
3    b  two -0.181099 -0.43465


In [17]:
pieces = dict(list(df.groupby('key1')))
pieces['a']

,key1,key2,data1,data2
0,a,one,0.043317,-0.951015
1,a,two,-0.063972,-0.200997
4,a,one,-0.749230,-1.952671


In [18]:
pieces['b']

,key1,key2,data1,data2
2,b,one,1.269132,-1.33983
3,b,two,-0.181099,-0.43465


In [19]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [21]:
grouped = df.groupby(df.dtypes, axis = 1) #Split the data by datatype

for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  0.043317 -0.951015
1 -0.063972 -0.200997
2  1.269132 -1.339830
3 -0.181099 -0.434650
4 -0.749230 -1.952671
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


## Selecting a Column or Subset of Columns

In [22]:
#identical to df['data2'].groupby(['key1','key2'])
df.groupby(['key1', 'key2'])[['data2']].mean() 

data2
key1 key2          
a    one  -1.451843
     two  -0.200997
b    one  -1.339830
     two  -0.434650

In [30]:
grouped = df['data2'].groupby([df['key1'],df['key2']]).mean()
grouped

key1  key2
a     one    -1.451843
      two    -0.200997
b     one    -1.339830
      two    -0.434650
Name: data2, dtype: float64

In [31]:
s_grouped = df.groupby(['key1','key2'])['data2']
s_grouped

In [32]:
s_grouped.mean()

key1  key2
a     one    -1.451843
      two    -0.200997
b     one    -1.339830
      two    -0.434650
Name: data2, dtype: float64

## Grouping with Dicts and Series

In [33]:
people = pd.DataFrame(np.random.randn(5, 5),
 ....: columns=['a', 'b', 'c', 'd', 'e'],
 ....: index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

people.iloc[2:3, [1, 2]] = np.nan

people

,a,b,c,d,e
Joe,0.343556,0.819318,0.242594,-0.185177,-0.859153
Steve,0.144618,-1.010506,0.113351,1.612961,-0.037607
Wes,0.399787,NaN,NaN,-0.203089,-0.117079
Jim,0.080532,-0.659769,0.400338,1.017876,0.357026
Travis,-0.018977,0.510838,0.317148,-0.967031,-1.065852


In [36]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
 ....: 'd': 'blue', 'e': 'red', 'f' : 'orange'}

by_column = people.groupby(mapping, axis = 1)
by_column.sum()

,blue,red
Joe,0.057417,0.303721
Steve,1.726312,-0.903494
Wes,-0.203089,0.282708
Jim,1.418214,-0.222212
Travis,-0.649883,-0.573991


In [37]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [38]:
people.groupby(map_series, axis = 1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


In [39]:
people.groupby(len).sum() #Group by the length of the name

,a,b,c,d,e
3,0.823875,0.159548,0.642932,0.629610,-0.619206
5,0.144618,-1.010506,0.113351,1.612961,-0.037607
6,-0.018977,0.510838,0.317148,-0.967031,-1.065852


In [40]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()


a         b         c         d         e
3 one  0.343556  0.819318  0.242594 -0.203089 -0.859153
  two  0.080532 -0.659769  0.400338  1.017876  0.357026
5 one  0.144618 -1.010506  0.113351  1.612961 -0.037607
6 two -0.018977  0.510838  0.317148 -0.967031 -1.065852

## Grouping by Index Levels

In [41]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
 ....: [1, 3, 5, 1, 3]],
 ....: names=['cty', 'tenor'])

columns

MultiIndex([('US', 1),
            ('US', 3),
            ('US', 5),
            ('JP', 1),
            ('JP', 3)],
           names=['cty', 'tenor'])

In [42]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.594747  0.988544  1.390311  0.736080 -0.814722
1      1.797609 -0.915162 -1.000432 -0.263730  0.538696
2      0.654919 -0.159555  0.751423 -0.991283  0.802062
3     -1.242143 -1.161044 -0.513562  1.039927 -1.183025

In [43]:
 hier_df.groupby(level='cty', axis=1).count() #Group by the label / index 

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## Data Aggregation

In [44]:
df

,key1,key2,data1,data2
0,a,one,0.043317,-0.951015
1,a,two,-0.063972,-0.200997
2,b,one,1.269132,-1.339830
3,b,two,-0.181099,-0.434650
4,a,one,-0.749230,-1.952671


In [45]:
grouped = df.groupby('key1')

In [47]:
grouped['data1'].quantile(0.9) #Value at 90th quantile

key1
a    0.021859
b    1.124109
Name: data1, dtype: float64

In [50]:
def peak_to_peak(arr):
    return arr.max() - arr.min() #Calculate range

grouped.agg(peak_to_peak) #Range for each group

,data1,data2
key1,,
a,0.792547,1.751673
b,1.450231,0.905180


In [51]:
 grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.256628  0.429965 -0.749230 -0.406601 -0.063972 -0.010327   
b      2.0  0.544016  1.025468 -0.181099  0.181459  0.544016  0.906574   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.043317   3.0 -1.034894  0.878844 -1.952671 -1.451843 -0.951015   
b     1.269132   2.0 -0.887240  0.640059 -1.339830 -1.113535 -0.887240   

                          
           75%       max  
key1                      
a    -0.576006 -0.200997  
b    -0.660945 -0.434650

## Column-Wise and Multiple Function Application

In [15]:
tips = pd.read_csv('examples/tips.csv') #Read csv
tips['tips_pct'] = tips['tip'] / tips['total_bill'] #Calculate tip percantage of total bill
tips[:6]

,total_bill,tip,smoker,day,time,size,tips_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [5]:
grouped = tips.groupby(['day','smoker']) #Split by day and smoker
grouped_pct = grouped['tips_pct'] #Take tip percentage data
grouped_pct.agg('mean') #Calulcate mean

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tips_pct, dtype: float64

In [10]:
def peaktopeak(x):
    return x.max() - x.min()

grouped_pct.agg(['mean', 'std', peaktopeak]) #Calculate mean, standard deviation and range

mean       std  peaktopeak
day  smoker                                
Fri  No      0.151650  0.028123    0.067349
     Yes     0.174783  0.051293    0.159925
Sat  No      0.158048  0.039767    0.235193
     Yes     0.147906  0.061375    0.290095
Sun  No      0.160113  0.042347    0.193226
     Yes     0.187250  0.154134    0.644685
Thur No      0.160298  0.038774    0.193350
     Yes     0.163863  0.039389    0.151240

In [11]:
#Foo becomes column names for mean and bar becomes column names for standard deviation 
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)]) 

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [14]:
functions = ['count', 'mean', 'max'] # List to hold functions to carry out
result = grouped['tips_pct','total_bill'].agg(functions)
result

tips_pct                     total_bill                  
               count      mean       max      count       mean    max
day  smoker                                                          
Fri  No            4  0.151650  0.187735          4  18.420000  22.75
     Yes          15  0.174783  0.263480         15  16.813333  40.17
Sat  No           45  0.158048  0.291990         45  19.661778  48.33
     Yes          42  0.147906  0.325733         42  21.276667  50.81
Sun  No           57  0.160113  0.252672         57  20.506667  48.17
     Yes          19  0.187250  0.710345         19  24.120000  45.35
Thur No           45  0.160298  0.266312         45  17.113111  41.19
     Yes          17  0.163863  0.241255         17  19.190588  43.11

In [16]:
result['tips_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [18]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)] #Set column names 
grouped['tips_pct', 'total_bill'].agg(ftuples)

tips_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [19]:
grouped.agg({'tip' : np.max, 'size' : 'sum'}) #Perform different functions based on the column

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [21]:
 grouped.agg({'tips_pct' : ['min', 'max', 'mean', 'std'], 'size' : 'sum'})

tips_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

In [23]:
tips.groupby(['day', 'smoker'], as_index=False).mean() #Remove row indexes

,day,smoker,total_bill,tip,size,tips_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## Apply

In [28]:
def top(df, n=5, column = 'tips_pct'):
    return df.sort_values(by=column)[-n:]

top(tips, n=6) #Top 5 tip percantages in ascending order

,total_bill,tip,smoker,day,time,size,tips_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [29]:
 tips.groupby('smoker').apply(top) #Split by smoker or not then calculate top 5 percentages

total_bill   tip smoker   day    time  size  tips_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [30]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size  tips_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [33]:
result = tips.groupby('smoker')['tips_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [34]:
result.unstack()

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [35]:
 tips.groupby('smoker', group_keys=False).apply(top) #Remove hierarchical indexing

,total_bill,tip,smoker,day,time,size,tips_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


## Quantile and Bucket Analysis

In [39]:
frame = pd.DataFrame({'data1': np.random.randn(1000), 'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4) #Split into  4 buckets/quartiles
quartiles[:10]

0     (0.152, 1.852]
1     (0.152, 1.852]
2    (-1.549, 0.152]
3     (1.852, 3.553]
4     (0.152, 1.852]
5     (0.152, 1.852]
6     (1.852, 3.553]
7     (0.152, 1.852]
8    (-1.549, 0.152]
9    (-1.549, 0.152]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.257, -1.549] < (-1.549, 0.152] < (0.152, 1.852] < (1.852, 3.553]]

In [43]:
def get_stats(group):
    return {'min':group.min(), 'max': group.max(), 'count': group.count(), 'mean': group.mean()}
grouped = frame.data2.groupby(quartiles) #Split data2 into the same quartiles
grouped.apply(get_stats).unstack() #Calculate statistscs for each quartile

,min,max,count,mean
data1,,,,
"(-3.257, -1.549]",-2.518638,2.922050,73.0,-0.090536
"(-1.549, 0.152]",-3.265965,3.216877,485.0,-0.061683
"(0.152, 1.852]",-2.660271,2.562205,414.0,-0.049237
"(1.852, 3.553]",-2.084408,2.485013,28.0,0.224127


In [44]:
grouping = pd.qcut(frame.data1, 10, labels = False) #10 equal length buckets
grouped = frame.data2.groupby(grouping) #Split into the buckets/bins
grouped.apply(get_stats).unstack() #Get the summary statistics

,min,max,count,mean
data1,,,,
0,-2.518638,2.922050,100.0,-0.088460
1,-3.265965,2.836878,100.0,-0.068061
2,-2.097204,3.216877,100.0,-0.131412
3,-2.178987,2.327266,100.0,0.111835
4,-2.636694,1.816769,100.0,-0.127846
5,-2.634108,2.082623,100.0,-0.179167
6,-2.575625,2.076591,100.0,0.019813
7,-2.660271,1.920804,100.0,-0.165109
8,-2.381378,2.562205,100.0,0.067245


## Filling Missing Values with Group-Specific Values

In [47]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1    0.748775
2         NaN
3   -1.032704
4         NaN
5    0.931853
dtype: float64

In [48]:
s.fillna(s.mean()) #Replace NA with the mean value

0    0.215975
1    0.748775
2    0.215975
3   -1.032704
4    0.215975
5    0.931853
dtype: float64

In [50]:
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index = states) #Assign states to each row
data

Ohio         -0.123107
New York      2.019761
Vermont      -0.783923
Florida       0.456756
Oregon       -0.387591
Nevada       -0.534489
California    0.602458
Idaho        -0.151485
dtype: float64

In [52]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan #Set missing values
data

Ohio         -0.123107
New York      2.019761
Vermont            NaN
Florida       0.456756
Oregon       -0.387591
Nevada             NaN
California    0.602458
Idaho              NaN
dtype: float64

In [53]:
data.groupby(group_key).mean() #Set the first 4 states as east and the last 4 as west then calculate the mean

East    0.784470
West    0.107433
dtype: float64

In [57]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio         -0.123107
New York      2.019761
Vermont       0.784470
Florida       0.456756
Oregon       -0.387591
Nevada        0.107433
California    0.602458
Idaho         0.107433
dtype: float64

In [58]:
fill_values = {'East': 0.5, 'West': -1}
fill_funct = lambda g: g.fillna(fill_values[g.name]) #Fill the na values by specific value
data.groupby(group_key).apply(fill_funct)

Ohio         -0.123107
New York      2.019761
Vermont       0.500000
Florida       0.456756
Oregon       -0.387591
Nevada       -1.000000
California    0.602458
Idaho        -1.000000
dtype: float64

## Example: Random Sampling and Permutation

In [9]:
#Constructing a deck of playing cards

suits = ['H', 'S', 'D', 'C'] #Card suits

card_val = (list(range(1,11)) + [10] *3) * 4 #1 for ace, 2-10 for number cards, 3 for picture cards (blackjack values)
base_names = ['A'] + list(range(2,11)) + ['J','K','Q'] #List for the prefix of each card
cards = []

for suit in suits:
    cards.extend(str(num) + suit for num in base_names) #Add the prefix to the suit

deck = pd.Series(card_val, index = cards) #Create series
deck[:13]         

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [10]:
def draw(deck, n=5):
    return deck.sample(n)

draw(deck) #Choose 5 random cards

KC    10
3S     3
6S     6
JH    10
5C     5
dtype: int64

In [12]:
get_suit = lambda x: x[-1] #Last letter of the card

#Group by the last letter of the cards(suit) then pick 2 from each category
deck.groupby(get_suit).apply(draw, n=2)

C  7C      7
   10C    10
D  2D      2
   5D      5
H  4H      4
   10H    10
S  AS      1
   6S      6
dtype: int64

## Group Weighted Average and Correlation

In [13]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
 .....: 'b', 'b', 'b', 'b'],
 .....: 'data': np.random.randn(8),
 .....: 'weights': np.random.rand(8)})

df

,category,data,weights
0,a,-0.384290,0.567011
1,a,-0.333659,0.144026
2,a,-1.860438,0.526494
3,a,-0.327863,0.023294
4,b,0.629540,0.700654
5,b,0.927828,0.448370
6,b,1.240612,0.089242
7,b,1.054419,0.891094


In [14]:
grouped = df.groupby('category') #Split by category
get_wavg = lambda g: np.average(g['data'], weights = g['weights']) #Calculate weighted average
grouped.apply(get_wavg) #Apply the average

category
a   -0.993872
b    0.895763
dtype: float64

In [5]:
close_px = pd.read_csv('examples/stock_px_2.csv', parse_dates = True, index_col = 0) #Ready csv
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


In [6]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [7]:
spx_corr = lambda x: x.corrwith(x['SPX']) #Calulcate correlation with SPX row value

rets = close_px.pct_change().dropna() #Caluate percentage change with previous row value

In [8]:
get_year = lambda x: x.year #Returns the year
by_year = close_px.groupby(get_year) #Split the data by the year
by_year.apply(spx_corr) #Calculate the correlation

,AAPL,MSFT,XOM,SPX
2003,0.882692,0.675932,0.860104,1.0
2004,0.638080,0.593300,0.432642,1.0
2005,0.743263,0.746188,0.404042,1.0
2006,0.838252,0.824727,0.849528,1.0
2007,0.483093,0.292706,0.729426,1.0
2008,0.829417,0.919476,0.774885,1.0
2009,0.952895,0.950194,0.278447,1.0
2010,0.617886,0.469280,0.894656,1.0
2011,-0.552395,0.111688,0.912603,1.0


In [10]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.666938
2004    0.784780
2005    0.570353
2006    0.815130
2007    0.761417
2008    0.729151
2009    0.951722
2010   -0.342094
2011    0.307734
dtype: float64

## Group-Wise Linear Regression

In [13]:
import statsmodels.api as sm

def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1
    result = sm.OLS(Y, X).fit() #Calculate least squares regression 
    return result.params

by_year.apply(regress, 'AAPL', ['SPX']) #Yearly linear regression 

,SPX,intercept
2003,0.018506,-8.589850
2004,0.132565,-132.121163
2005,0.286831,-299.595103
2006,0.155668,-133.186913
2007,0.379906,-432.917550
2008,0.146157,-36.338143
2009,0.328253,-164.384872
2010,0.412900,-210.849814
2011,-0.193834,603.876911


##  Pivot Tables and Cross-Tabulation


In [16]:
tips.pivot_table(index = ['day', 'smoker']) #Create pivot table

size       tip  tips_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [17]:
#Puts size and tips_pct in the columns and the time and day in the rows
tips.pivot_table(['tips_pct', 'size'], index = ['time','day'], columns = 'smoker')

size            tips_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [19]:
#Give the mean of each categroy, all means include both no and yes columns' data
tips.pivot_table(['tips_pct', 'size'], index=['time', 'day'], columns='smoker', margins=True)

size                      tips_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [21]:
#Gives a count of all the different categories
tips.pivot_table('tips_pct', index=['time', 'smoker'], columns='day', aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [22]:
#Fill NA vlaues with 0 
tips.pivot_table('tips_pct', index=['time', 'size', 'smoker'],  columns='day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706